In [2]:
import numpy as np

from config import Configs
from models.TSSCD import TSSCD_FCN, TSSCD_Unet
from utils import *
from data_loader import load_data

In [2]:
npz_file = np.load(os.path.join(f'.\\TimeSeriesImages\\SA_blocks_clipped&mosaic\\ZJ_data4cls', f'block_5755.npz'))['data']

In [3]:
device = device_on()
configs = Configs()
model = TSSCD_Unet(configs.input_channels, configs.classes, configs.model_hidden)
model = model.to(device)
model_state_dict = torch.load(os.path.join('model_data\\TSSCD_Unet', '6.pth'), map_location='cuda', weights_only=True)
model.load_state_dict(model_state_dict)
model.eval()

Uisng cuda device


TSSCD_Unet(
  (layer1): Sequential(
    (0): Conv1d(12, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU(inplace=True)
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (1): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (2): ReLU(inplace=True)
    (3): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU(inplace=True)
  )
  (layer3): Sequential(
    (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (1): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (2): ReLU(inplace=True)
    (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (1): Conv1d(512, 1024, kernel_size

In [4]:
valid_data = np.load('./dataset/test.npy')
print(valid_data.shape) # counts, channel, seq_length
print(valid_data[114, -1, :])

(748, 13, 60)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
valid_data[:, :-1, :] = minMax_standardization(valid_data[:, :-1, :])
print(valid_data[:, :-1, :].shape)

(748, 12, 60)


In [ ]:
samp_idx = 552
# samp_idx = 227
model_input = torch.Tensor(valid_data[samp_idx:samp_idx+1, :-1, :]).to(device)

label = valid_data[samp_idx:samp_idx+1, -1, :].astype(int)
pres = torch.argmax(input=model(model_input), dim=1).cpu().numpy()

print(pres[0], label[0], sep='\n')
# ???

[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [7]:
labels = valid_data[samp_idx:samp_idx+1, -1, :].astype(int)
pres = torch.argmax(input=model(model_input), dim=1).cpu().numpy()

from metrics import Evaluator, SpatialChangeDetectScore, TemporalChangeDetectScore
spatialscore = SpatialChangeDetectScore()
temporalscore = TemporalChangeDetectScore(series_length=60, error_rate=1)

for pre, label in zip(pres, labels):
    pre, label = pre[None, :], label[None, :]
    predata, prechangepoints, pretypes = FilteringSeries(pre, method='Majority', window_size=5)
    labdata, labchangepoints, labtypes = FilteringSeries(label)
    
    spatialscore.addValue(labchangepoints[0], prechangepoints[0])
    temporalscore.addValue(labchangepoints[0], prechangepoints[0])
    spatialscore.addLccValue(pretypes[0], labtypes[0])
    
    print(f'Lcc accuracy: {spatialscore.getLccScore()}')
    print(f'label change points\t: {labchangepoints[0]}', f'predicted change points\t: {prechangepoints[0]}', sep='\n')
    print(f'PreChange_LabChange: {temporalscore.PreChange_LabChange}',
          f'PreNoChange_LabChange: {temporalscore.PreNoChange_LabChange}',
          f'PreChange_LabNoChange: {temporalscore.PreChange_LabNoChange}', 
          f'PreNoChange_LabNoChange: {temporalscore.PreNoChange_LabNoChange}',
          sep='\n')

org pre: [11 26 32 36]
better_pre: [11 26 33 36]
label: [11 26 33 36]
hot_pre: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
hot_label: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lcc accuracy: 1.0
label change points	: [11 26 33 36]
predicted change points	: [11 26 33 36]
PreChange_LabChange: 4.0000001192092896
PreNoChange_LabChange: 1.1920928955078125e-07
PreChange_LabNoChange: 1.1920928955078125e-07
PreNoChange_LabNoChange: 56.00000011920929


In [3]:
import numpy as np
from utils import spatial_mode_filter

test_matrix = np.random.randint(0, 10, (5,5,3))  # 直接生成5x5x3矩阵

print("原始矩阵维度:", test_matrix.shape)
print("\n时间维度1（滤波前）:")
print(test_matrix[:, :, 0])
# print("\n时间维度2（滤波前）:")
# print(test_matrix[:, :, 1])
# print("\n时间维度3（滤波前）:")
# print(test_matrix[:, :, 2])

# 应用3x3空间众数滤波
filtered = spatial_mode_filter(test_matrix, kernel_size=3)

print("\n滤波后结果:")
print("时间维度1（高频噪声滤波）:")
print(filtered[:, :, 0])
# print("\n时间维度2（中心模式保持）:")
# print(filtered[:, :, 1])
# print("\n时间维度3（边界填充测试）:")
# print(filtered[:, :, 2])

原始矩阵维度: (5, 5, 3)

时间维度1（滤波前）:
[[2 1 3 3 4]
 [5 7 4 5 7]
 [0 2 1 2 6]
 [8 6 9 5 0]
 [5 8 3 8 9]]


RuntimeError: boundary mode not supported